In [1]:
from pathlib import Path
import sys
import os 

PROJECT_ROOT = Path.cwd()
while PROJECT_ROOT.name != "Thesis code" and PROJECT_ROOT != PROJECT_ROOT.parent:
    PROJECT_ROOT = PROJECT_ROOT.parent

sys.path.insert(0, str(PROJECT_ROOT))

from Functions.data_utils import (
    plot_incremental_response_rate,
    uplift_by_decile_bin,
    coerce_metrics_to_numeric,
)

C:\Users\tsterk\AppData\Local\anaconda3\envs\causalml_py312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to import duecredit due to No module named 'duecredit'


In [2]:
from __future__ import annotations

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.calibration import CalibratedClassifierCV
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    StratifiedShuffleSplit,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight

from catboost import CatBoostClassifier

from dirichletcal.calib.fulldirichlet import FullDirichletCalibrator
from sklearn.base import clone
#import calibration
from sklearn.utils import resample
from classifier_calibration.calibration_error import classwise_ece
import time
from typing import Tuple, Union
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd
import math
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [3]:
file_path = r"Data/covariates_modeling_uplift_models_2026-02-28.csv"
df = pd.read_csv(file_path)


C:\Users\tsterk\AppData\Local\Temp\ipykernel_21856\4048377555.py:2: DtypeWarning: Columns (0: total_volume, 1: retail_sales, 2: food_total, 3: vhms_total, 4: sports_total, 5: beauty_total, 6: monetary_value_52wk, 7: online_sales_52w, 8: retail_sales_52w, 9: monetary_value_53w_104w, 10: online_sales_53w_104w, 11: retail_sales_53w_104w) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [4]:
df['cameback'].value_counts()

cameback
0    264223
1      3440
Name: count, dtype: int64

In [5]:
treatment_converter = {
    "BNLX_ChurnP_10_test_export.csv": 1,
    "BNLX_ChurnP_10_controle_export.csv": 0, 
    
    "BNLX_ChurnP_25_test_export.csv": 2,
    "BNLX_ChurnP_25_controle_export.csv": 0,

    "BNLX_ChurnP_5eu_test_export.csv": 3,
    "BNLX_ChurnP_5eu_controle_export.csv": 0,

    "BNLX_ChurnP_10eu_test_export.csv": 4,
    "BNLX_ChurnP_10eu_controle_export.csv": 0,
    
    "BNLX_ChurnP_250_test_export.csv": 5,
    "BNLX_ChurnP_250_controle_export.csv": 0,

    "BNLX_ChurnP_500_test_export.csv": 6,
    "BNLX_ChurnP_500_controle_export.csv": 0,    
    
    "BNLX_ChurnP_SKUd_test_export.csv": 7,
    "BNLX_ChurnP_SKUd_controle_export.csv": -99,

    "BNLX_ChurnP_SKUe_test_export.csv": 8,
    "BNLX_ChurnP_SKUe_controle_export.csv": 0,    
    
    "BNLX_ChurnP_niks_test_export.csv": 9,
    "BNLX_ChurnP_niks_controle_export.csv": 0,
}

df["treatment"] = df["treatment_indicator"].map(treatment_converter)


In [6]:
categorical_cols  = ['has_rfl','gender','country_sk']
numeric_cols = [ 'recency', 'frequency', 'monetary_value',  'total_volume', 'length_of_relationship', 'online_sales', 'retail_sales', 
      'food_total', 'vhms_total', 'sports_total', 'beauty_total', 'frequency_52wk', 'monetary_value_52wk', 'volume_52wk', 'online_sales_52w', 
       'retail_sales_52w', 'frequency_53w_104w', 'monetary_value_53w_104w', 'volume_53w_104w', 'online_sales_53w_104w','retail_sales_53w_104w']

In [7]:
df = coerce_metrics_to_numeric(df, numeric_cols)

df[categorical_cols] = df[categorical_cols].astype("object")
df[numeric_cols] = df[numeric_cols].astype("int64")


In [8]:
(100 * df.groupby('treatment_indicator')['cameback'].mean()).round(2)


treatment_indicator
BNLX_ChurnP_10_controle_export.csv      1.13
BNLX_ChurnP_10_test_export.csv          1.29
BNLX_ChurnP_10eu_controle_export.csv    1.28
BNLX_ChurnP_10eu_test_export.csv        1.49
BNLX_ChurnP_250_controle_export.csv     1.17
BNLX_ChurnP_250_test_export.csv         1.51
BNLX_ChurnP_25_controle_export.csv      1.16
BNLX_ChurnP_25_test_export.csv          1.28
BNLX_ChurnP_500_controle_export.csv     1.35
BNLX_ChurnP_500_test_export.csv         1.58
BNLX_ChurnP_5eu_controle_export.csv     1.11
BNLX_ChurnP_5eu_test_export.csv         1.42
BNLX_ChurnP_SKUd_controle_export.csv    1.19
BNLX_ChurnP_SKUd_test_export.csv        1.19
BNLX_ChurnP_SKUe_controle_export.csv    1.20
BNLX_ChurnP_SKUe_test_export.csv        1.27
BNLX_ChurnP_niks_controle_export.csv    1.23
BNLX_ChurnP_niks_test_export.csv        1.28
Name: cameback, dtype: float64

In [9]:
df["multi_outcome"] = np.where(
    df["cameback"].to_numpy() != 0,
    "cameback_" + df["treatment"].astype(str),
    "no_cameback_" + df["treatment"].astype(str),
)

df = df[df['multi_outcome'].isin(['cameback_9','no_cameback_9','cameback_8','no_cameback_8',
                                'cameback_6','no_cameback_6','cameback_5','no_cameback_5',
                                  'cameback_4','no_cameback_4','cameback_3','no_cameback_3',
                                  'cameback_2','no_cameback_2','cameback_1','no_cameback_1',
                                  'cameback_0','no_cameback_0'])]

y = df["multi_outcome"]
t = df["treatment"]
X = df.drop(columns=["multi_outcome", "treatment_indicator", 'treatment', "cameback", 'customer_nk'])


In [10]:
# Class weights
alpha = 0.9

In [11]:
def fill_cat_missing(df, cat_cols):
    df = df.copy()
    df[cat_cols] = df[cat_cols].astype("string").fillna("MISSING")
    return df

In [12]:
import threading
import time

def _start_fit_watchdog(label: str, every_s: int = 600):
    stop_event = threading.Event()
    t0 = time.perf_counter()

    def _worker():
        next_print = t0 + every_s
        while not stop_event.is_set():
            remaining = next_print - time.perf_counter()
            if remaining > 0:
                stop_event.wait(remaining)
            if stop_event.is_set():
                break

            elapsed = time.perf_counter() - t0
            print(f"[Dirichlet:{label}] still fitting (elapsed={elapsed/600:.1f} min)")
            next_print += every_s

    thread = threading.Thread(target=_worker, daemon=True)
    thread.start()
    return stop_event, t0


def _stop_fit_watchdog(stop_event, t0, label: str):
    stop_event.set()
    elapsed = time.perf_counter() - t0
    print(f"[Dirichlet:{label}] fit finished (elapsed={elapsed/600:.2f} min)")

In [13]:
y_series = pd.Series(y, index=X.index)
all_classes = np.unique(y_series)

# ── Cross-validation setup ─────────────────────────────────────────────────────
# Stratify on treatment × outcome to ensure each fold has balanced class/treatment
# representation across both the uplift labels and target variable
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_s = pd.Series(y, index=X.index).astype(str)
t_s = pd.Series(t, index=X.index).astype(str)
strata = t_s + "_" + y_s.astype(str)

fold_results_uncal = []
fold_results_cal = []
fold_results_cat = []
fold_results_cat_cal = []

for fold, (train_idx, test_idx) in enumerate(skf.split(X, strata), start=1):
    start_time = time.perf_counter()
    print(f"\n========== FOLD {fold} ==========")

    X_train = X.iloc[train_idx]
    X_test = X.iloc[test_idx]

    y_train = y_series.iloc[train_idx]
    y_test = y_series.iloc[test_idx]

    # ── Feature typing & missing value handling ────────────────────────────────
    # Separate numeric/bool features from categoricals; fill missing cat values
    # before passing to the OHE preprocessor
    num_features = X_train.select_dtypes(include=["number", "bool"]).columns.tolist()
    cat_features = [c for c in X_train.columns if c not in num_features]

    X_train = fill_cat_missing(X_train, cat_features)
    X_test = fill_cat_missing(X_test, cat_features)

    # ── Class weighting ────────────────────────────────────────────────────────
    # Compute balanced class weights and apply exponent alpha to soften/sharpen
    # the penalty — helps counter the heavy class imbalance in churn labels
    class_labels = np.unique(y_train)
    class_weights = compute_class_weight(
        class_weight="balanced",
        classes=class_labels,
        y=y_train,
    )
    class_weight_dict = {
        label: weight ** alpha
        for label, weight in zip(class_labels, class_weights)
    }

    # ── Preprocessing pipeline ─────────────────────────────────────────────────
    # Numeric features passed through as-is; categoricals one-hot encoded
    # Unknown categories at test time are silently ignored
    preprocess = ColumnTransformer(
        transformers=[
            ("num", "passthrough", num_features),
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
        ],
        remainder="drop",
    )

    # ══════════════════════════════════════════════════════════════════════════
    # BLOCK 1 — Random Forest (no calibration)
    # Train RF on the full training fold and generate raw softmax probabilities.
    # These serve as the uncalibrated baseline for comparison.
    # ══════════════════════════════════════════════════════════════════════════
    rf = Pipeline(
        steps=[
            ("prep", preprocess),
            ("rf", RandomForestClassifier(
                random_state=42,
                n_jobs=-1,
                class_weight=class_weight_dict,
            )),
        ]
    )

    rf.fit(X_train, y_train)

    proba_test = rf.predict_proba(X_test)
    fold_classes = rf.named_steps["rf"].classes_

    y_pred = fold_classes[np.argmax(proba_test, axis=1)]
    confidence = np.max(proba_test, axis=1)

    fold_df_uncal = (
        pd.DataFrame(
            proba_test,
            index=X_test.index,
            columns=[f"p_{c}" for c in fold_classes],
        )
        .assign(
            y_true=y_test.to_numpy(),
            y_pred=y_pred,
            confidence=confidence,
            model="rf_uncal",
            fold=fold,
        )
    )
    fold_results_uncal.append(fold_df_uncal)

    # ══════════════════════════════════════════════════════════════════════════
    # BLOCK 2 — Random Forest + Dirichlet calibration
    # Re-train RF on a 70% sub-split, then fit a FullDirichletCalibrator on the
    # held-out 30% calibration set. The calibration set is oversampled to be
    # class-balanced before fitting, to prevent Dirichlet collapsing predictions
    # into the majority class.
    # ══════════════════════════════════════════════════════════════════════════

    # Split training fold into fit / calibration subsets, stratified on
    # treatment × outcome to preserve label balance in both halves
    t_train = t_s.iloc[train_idx]
    strata_train = t_train + "_" + y_train.astype(str)

    X_fit, X_calib, y_fit, y_calib, t_fit, t_calib = train_test_split(
        X_train,
        y_train,
        t_train,
        test_size=0.3,
        stratify=strata_train,
        random_state=42,
    )

    rf.fit(X_fit, y_fit)
    cal_classes = rf.named_steps["rf"].classes_

    # Get calibration probabilities and encode labels as integer codes
    p_cal = rf.predict_proba(X_calib)
    class_to_idx = {c: i for i, c in enumerate(cal_classes)}

    y_calib_codes = (
        pd.Series(y_calib, index=X_calib.index)
          .map(class_to_idx)
          .to_numpy()
    )

    # Oversample minority classes in the calibration set so Dirichlet sees a
    # balanced label distribution — avoids degenerate majority-class collapse
    temp_df = pd.DataFrame(p_cal)
    temp_df['target'] = y_calib_codes

    max_class_count = pd.Series(y_calib_codes).value_counts().max()

    balanced_df = pd.concat([
        resample(temp_df[temp_df['target'] == c],
                 replace=True,
                 n_samples=max_class_count,
                 random_state=42)
        for c in np.unique(y_calib_codes)
    ])

    p_cal_balanced = balanced_df.drop('target', axis=1).to_numpy()
    y_cal_balanced = balanced_df['target'].to_numpy()

    # Fit Dirichlet calibrator; watchdog logs progress every 60s in case of
    # slow convergence with fmin_l_bfgs_b
    print(f"[Dirichlet:RF fold={fold}] reg_lambda=0.1 optimizer=fmin_l_bfgs_b")
    print(f"[Dirichlet:RF fold={fold}] X shape={p_cal_balanced.shape} y shape={y_cal_balanced.shape}")
    print(f"[Dirichlet:RF fold={fold}] y counts: {pd.Series(y_cal_balanced).value_counts().to_dict()}")
    stop_evt, t0 = _start_fit_watchdog(label=f"RF fold={fold}", every_s=60)
    try:
        cal = FullDirichletCalibrator(optimizer="fmin_l_bfgs_b", reg_lambda=0.1)
        cal.fit(p_cal_balanced, y_cal_balanced)
    finally:
        _stop_fit_watchdog(stop_evt, t0, label=f"RF fold={fold}")

    # Apply calibrator to raw test probabilities
    p_test_raw = rf.predict_proba(X_test)
    proba_test_cal = cal.predict_proba(p_test_raw)

    y_pred_cal = cal_classes[np.argmax(proba_test_cal, axis=1)]
    confidence_cal = np.max(proba_test_cal, axis=1)

    print("RF cal y_pred dist:\n", pd.Series(y_pred_cal).value_counts(normalize=True))
    print("RF mean proba (cal):", proba_test_cal.mean(axis=0))

    fold_df_cal = (
        pd.DataFrame(
            proba_test_cal,
            index=X_test.index,
            columns=[f"p_{c}" for c in cal_classes],
        )
        .assign(
            y_true=y_test.to_numpy(),
            y_pred=y_pred_cal,
            confidence=confidence_cal,
            model="rf_calibrated_dirichlet",
            fold=fold,
        )
    )
    fold_results_cal.append(fold_df_cal)

    # ══════════════════════════════════════════════════════════════════════════
    # BLOCK 3 — CatBoost (no calibration)
    # Train CatBoost on the full training fold using native categorical feature
    # handling (no OHE needed). Probabilities are aligned to the global class
    # order (all_classes) so folds can be safely concatenated later.
    # ══════════════════════════════════════════════════════════════════════════
    class_names = list(class_labels)

    cat = CatBoostClassifier(
        verbose=100,
        random_seed=42,
        class_names=class_names,
        class_weights=[class_weight_dict[c] for c in class_labels],
        loss_function="MultiClass",
        thread_count=-1,
    )

    cat.fit(
        X_train,
        y_train,
        cat_features=cat_features,
    )

    proba_cat_test = cat.predict_proba(X_test)
    cat_classes = cat.classes_

    # Re-align CatBoost class probabilities to the global all_classes ordering
    # since CatBoost may return classes in a different order per fold
    proba_cat_aligned = np.zeros((len(test_idx), len(all_classes)), dtype=float)
    for j, c in enumerate(cat_classes):
        proba_cat_aligned[:, np.where(all_classes == c)[0][0]] = proba_cat_test[:, j]

    y_pred_cat = cat_classes[np.argmax(proba_cat_test, axis=1)]
    confidence_cat = np.max(proba_cat_test, axis=1)

    fold_df_cat = (
        pd.DataFrame(
            proba_cat_aligned,
            index=X_test.index,
            columns=[f"p_{c}" for c in all_classes],
        )
        .assign(
            y_true=y_test.to_numpy(),
            y_pred=y_pred_cat,
            confidence=confidence_cat,
            model="catboost_uncal",
            fold=fold,
        )
    )
    fold_results_cat.append(fold_df_cat)

    # ══════════════════════════════════════════════════════════════════════════
    # BLOCK 4 — CatBoost + Dirichlet calibration
    # Mirrors the RF calibration approach: re-train on the 70% fit split,
    # calibrate on the oversampled 30% hold-out, then apply to test set.
    # Watchdog interval is longer (600s) as CatBoost calibration can be slower.
    # ══════════════════════════════════════════════════════════════════════════

    # Re-train CatBoost on the fit subset (same split used for RF calibration)
    cat.fit(X_fit, y_fit, cat_features=cat_features)
    cal_classes = cat.classes_

    # Encode calibration labels as integer codes aligned to CatBoost class order
    proba_calib = cat.predict_proba(X_calib)
    y_calib_codes = pd.Categorical(y_calib, categories=cal_classes).codes

    # Oversample calibration set to balanced class counts (same logic as RF above)
    temp_df = pd.DataFrame(proba_calib)
    temp_df['target'] = y_calib_codes

    max_class_count = pd.Series(y_calib_codes).value_counts().max()

    balanced_df = pd.concat([
        resample(temp_df[temp_df['target'] == c],
                 replace=True,
                 n_samples=max_class_count,
                 random_state=42)
        for c in np.unique(y_calib_codes)
    ])

    proba_calib_bal = balanced_df.drop('target', axis=1).to_numpy()
    y_cal_bal = balanced_df['target'].to_numpy()

    # Fit Dirichlet calibrator; watchdog set to 600s given CatBoost's longer
    # calibration convergence time
    print(f"[Dirichlet:CAT fold={fold}] reg_lambda=0.1 optimizer=fmin_l_bfgs_b")
    print(f"[Dirichlet:CAT fold={fold}] X shape={proba_calib_bal.shape} y shape={y_cal_bal.shape}")
    print(f"[Dirichlet:CAT fold={fold}] y counts: {pd.Series(y_cal_bal).value_counts().to_dict()}")
    stop_evt, t0 = _start_fit_watchdog(label=f"CAT fold={fold}", every_s=600)
    try:
        cal = FullDirichletCalibrator(optimizer="fmin_l_bfgs_b", reg_lambda=0.1)
        cal.fit(proba_calib_bal, y_cal_bal)
    finally:
        _stop_fit_watchdog(stop_evt, t0, label=f"CAT fold={fold}")

    # Apply calibrator to raw CatBoost test probabilities
    proba_test_raw = cat.predict_proba(X_test)
    proba_test_cal = cal.predict_proba(proba_test_raw)

    y_pred_cal = cal_classes[np.argmax(proba_test_cal, axis=1)]
    confidence_cal = np.max(proba_test_cal, axis=1)

    print("Cat cal y_pred dist:\n", pd.Series(y_pred_cal).value_counts(normalize=True))
    print("Cat mean proba (cal):", proba_test_cal.mean(axis=0))

    fold_df_cat_cal = (
        pd.DataFrame(
            proba_test_cal,
            index=X_test.index,
            columns=[f"p_{c}" for c in cal_classes],
        )
        .assign(
            y_true=y_test.to_numpy(),
            y_pred=y_pred_cal,
            confidence=confidence_cal,
            model="catboost_calibrated_dirichlet",
            fold=fold,
        )
    )
    fold_results_cat_cal.append(fold_df_cat_cal)

    elapsed = time.perf_counter() - start_time
    print(f"FOLD {fold} took {elapsed:.2f} seconds")

# ── Combine fold results & restore original index order ───────────────────────
# .loc[X.index] re-sorts rows back to the original dataset order after
# concatenating out-of-order fold predictions
df_uncalibrated_rf = pd.concat(fold_results_uncal, axis=0).loc[X.index]
df_calibrated_rf_dirichlet = pd.concat(fold_results_cal, axis=0).loc[X.index]
df_uncalibrated_catboost = pd.concat(fold_results_cat, axis=0).loc[X.index]
df_calibrated_catboost_dirichlet = pd.concat(fold_results_cat_cal, axis=0).loc[X.index]


========== FOLD 1 ==========
[Dirichlet:RF fold=1] reg_lambda=0.1 optimizer=fmin_l_bfgs_b
[Dirichlet:RF fold=1] X shape=(512334, 18) y shape=(512334,)
[Dirichlet:RF fold=1] y counts: {0: 28463, 1: 28463, 2: 28463, 3: 28463, 4: 28463, 5: 28463, 6: 28463, 7: 28463, 8: 28463, 9: 28463, 10: 28463, 11: 28463, 12: 28463, 13: 28463, 14: 28463, 15: 28463, 16: 28463, 17: 28463}
[Dirichlet:RF fold=1] still fitting (elapsed=0.1 min)
[Dirichlet:RF fold=1] still fitting (elapsed=0.2 min)
[Dirichlet:RF fold=1] still fitting (elapsed=0.3 min)
[Dirichlet:RF fold=1] still fitting (elapsed=0.4 min)
[Dirichlet:RF fold=1] still fitting (elapsed=0.5 min)
[Dirichlet:RF fold=1] still fitting (elapsed=0.6 min)
[Dirichlet:RF fold=1] still fitting (elapsed=0.7 min)
[Dirichlet:RF fold=1] still fitting (elapsed=0.8 min)
[Dirichlet:RF fold=1] still fitting (elapsed=0.9 min)
[Dirichlet:RF fold=1] still fitting (elapsed=1.0 min)
[Dirichlet:RF fold=1] still fitting (elapsed=1.1 min)
[Dirichlet:RF fold=1] still fitti

In [14]:
df_calibrated_rf_dirichlet.head()

,p_cameback_0,p_cameback_1,p_cameback_2,p_cameback_3,p_cameback_4,p_cameback_5,p_cameback_6,p_cameback_8,p_cameback_9,p_no_cameback_0,...,p_no_cameback_4,p_no_cameback_5,p_no_cameback_6,p_no_cameback_8,p_no_cameback_9,y_true,y_pred,confidence,model,fold
0,0.069161,0.064780,0.057297,0.075996,0.067137,0.063920,0.070437,0.072442,0.040604,0.046373,...,0.045605,0.046602,0.044146,0.046189,0.050000,no_cameback_0,cameback_3,0.075996,rf_calibrated_dirichlet,1
2,0.062543,0.038095,0.072894,0.076475,0.085783,0.055029,0.103968,0.170394,0.007307,0.034275,...,0.041043,0.031218,0.037359,0.038011,0.038569,no_cameback_8,cameback_8,0.170394,rf_calibrated_dirichlet,2
3,0.043599,0.064911,0.090859,0.100361,0.094233,0.032092,0.167313,0.101776,0.019439,0.029667,...,0.038194,0.028751,0.032717,0.031722,0.032712,cameback_9,cameback_6,0.167313,rf_calibrated_dirichlet,2
4,0.051952,0.079701,0.023806,0.048930,0.056595,0.125568,0.111180,0.045378,0.014688,0.049986,...,0.058517,0.043854,0.050085,0.045972,0.054365,no_cameback_0,cameback_5,0.125568,rf_calibrated_dirichlet,4
5,0.043693,0.058030,0.053906,0.010765,0.030522,0.092992,0.062028,0.029683,0.041500,0.062907,...,0.056219,0.070059,0.068983,0.063597,0.063998,no_cameback_0,cameback_5,0.092992,rf_calibrated_dirichlet,1


In [15]:
df_preds = pd.concat(
    [
        df_uncalibrated_rf,
        df_calibrated_rf_dirichlet,
         df_uncalibrated_catboost,
         df_calibrated_catboost_dirichlet
    ],
    ignore_index=True,
)

df_preds = df_preds.assign(correct=df_preds["y_pred"] == df_preds["y_true"])


In [16]:
# Saving DF for uplift analytics
def save_df_preds(df_preds, path: str = "Data/df_preds.csv") -> None:
    df_preds.to_csv(path, index=False)
save_df_preds(df_preds)

## Diagnostics calibration and performance per class

In [17]:
from sklearn.metrics import precision_score, recall_score
import pandas as pd

def metrics_per_model_per_class(df: pd.DataFrame) -> pd.DataFrame:
    labels = (
        [f"cameback_{i}" for i in range(10)] +
        [f"no_cameback_{i}" for i in range(10)]
    )
    
    rows = []
    for model, group in df.groupby("model"):
        prec = precision_score(group["y_true"], group["y_pred"], labels=labels, average=None, zero_division=0)
        rec = recall_score(group["y_true"], group["y_pred"], labels=labels, average=None, zero_division=0)
        
        for i, label in enumerate(labels):
            rows.append({
                "model": model,
                "class": label,
                "precision": prec[i],
                "recall": rec[i],
            })
    
    return pd.DataFrame(rows)

metrics_per_model_per_class(df_preds)

,model,class,precision,recall
0,catboost_calibrated_dirichlet,cameback_0,0.015652,0.012474
1,catboost_calibrated_dirichlet,cameback_1,0.000868,0.109948
2,catboost_calibrated_dirichlet,cameback_2,0.000954,0.102041
3,catboost_calibrated_dirichlet,cameback_3,0.000599,0.047619
4,catboost_calibrated_dirichlet,cameback_4,0.001453,0.201794
...,...,...,...,...
75,rf_uncal,no_cameback_5,0.077469,0.018031
76,rf_uncal,no_cameback_6,0.066054,0.015612
77,rf_uncal,no_cameback_7,0.000000,0.000000
78,rf_uncal,no_cameback_8,0.057246,0.013158


In [18]:
import numpy as np
import pandas as pd
from classifier_calibration.calibration_error import classwise_ece
proba_cols = [
    "p_cameback_0",
    "p_cameback_1",
    "p_cameback_2",
    "p_cameback_3",
    "p_cameback_4",
    "p_cameback_5",
    "p_cameback_6",
    "p_cameback_8",
    "p_cameback_9",
    "p_no_cameback_0",
    "p_no_cameback_1",
    "p_no_cameback_2",
    "p_no_cameback_3",
    "p_no_cameback_4",
    "p_no_cameback_5",
    "p_no_cameback_6",
    "p_no_cameback_8",
    "p_no_cameback_9",
]

class_mapping = {
    "cameback_0": 0,
    "cameback_1": 1,
    "cameback_2": 2,
    "cameback_3": 3,
    "cameback_4": 4,
    "cameback_5": 5,
    "cameback_6": 6,
    "cameback_8": 7,
    "cameback_9": 8,
    "no_cameback_0": 9,
    "no_cameback_1": 10,
    "no_cameback_2": 11,
    "no_cameback_3": 12,
    "no_cameback_4": 13,
    "no_cameback_5": 14,
    "no_cameback_6": 15,
    "no_cameback_8": 16,
    "no_cameback_9": 17,
}
def compute_classwise_ece_per_model(df: pd.DataFrame) -> pd.DataFrame:
    rows = []

    for model, g in df.groupby("model"):
        y_prob = g[proba_cols].to_numpy()
        y_true_int = g["y_true"].map(class_mapping).to_numpy()

        if np.isnan(y_true_int).any():
            bad_labels = (
                g.loc[np.isnan(y_true_int), "y_true"]
                .unique()
                .tolist()
            )
            raise ValueError(f"Unmapped y_true labels found for model {model}: {bad_labels}")

        y_true_int = y_true_int.astype(int)

        ece_per_class = classwise_ece(
            labels=y_true_int,
            pred_probs=y_prob
        )

        rows.append(
            {
                "model": model,
                "classwise_ece": float(np.mean(ece_per_class)),
            }
        )

    return pd.DataFrame(rows)


ece_df = compute_classwise_ece_per_model(df_preds)
ece_df


,model,classwise_ece
0,catboost_calibrated_dirichlet,0.051374
1,catboost_uncal,0.042498
2,rf_calibrated_dirichlet,0.052691
3,rf_uncal,0.024549


In [19]:
df_preds[(df_preds['model']=='catboost_uncal') &(df_preds['p_cameback_3'] > .2)]['y_true']

478050    no_cameback_1
478164    no_cameback_0
478254    no_cameback_5
478441    no_cameback_0
478651    no_cameback_6
              ...      
715527    no_cameback_9
715976    no_cameback_6
716107    no_cameback_0
716216    no_cameback_0
716465    no_cameback_0
Name: y_true, Length: 940, dtype: str

In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import brier_score_loss, log_loss

def brier_and_logloss_per_model_class(
    df: pd.DataFrame,
    proba_cols: list[str] = proba_cols,
    class_mapping: dict[str, int] = class_mapping,
    eps: float = 1e-15,
) -> pd.DataFrame:
    class_labels = [c.replace("p_", "") for c in proba_cols]
    inv_class_mapping = {v: k for k, v in class_mapping.items()}

    rows: list[dict] = []

    for model, g in df.groupby("model"):
        y_true_int = g["y_true"].map(class_mapping).to_numpy()

        if np.isnan(y_true_int).any():
            bad_labels = (
                g.loc[np.isnan(y_true_int), "y_true"]
                .unique()
                .tolist()
            )
            raise ValueError(f"Unmapped y_true labels found for model {model}: {bad_labels}")

        y_true_int = y_true_int.astype(int)
        y_prob = g[proba_cols].to_numpy()

        model_logloss_values = []

        # Per-class metrics (one-vs-rest)
        for k in range(len(proba_cols)):
            y_true_bin = (y_true_int == k).astype(int)

            p_k = np.clip(y_prob[:, k], eps, 1 - eps)

            ll = float(
                log_loss(
                    y_true_bin,
                    np.column_stack([1 - p_k, p_k]),
                    labels=[0, 1],
                )
            )

            model_logloss_values.append(ll)

            rows.append(
                {
                    "model": model,
                    "class": inv_class_mapping[k],
                    "logloss": ll,
                }
            )

        # ---- Mean log loss row ----
        rows.append(
            {
                "model": model,
                "class": "mean",
                "logloss": float(np.mean(model_logloss_values)),
            }
        )

    return pd.DataFrame(rows)


metrics_df = brier_and_logloss_per_model_class(df_preds)
metrics_df[metrics_df['class']=='mean']


,model,class,logloss
18,catboost_calibrated_dirichlet,mean,0.212076
37,catboost_uncal,mean,0.180447
56,rf_calibrated_dirichlet,mean,0.212822
75,rf_uncal,mean,0.254258


In [21]:
import numpy as np
import pandas as pd
from sklearn.metrics import brier_score_loss, log_loss

def brier_and_logloss_per_model_class(
    df: pd.DataFrame,
    proba_cols: list[str] = proba_cols,
    class_mapping: dict[str, int] = class_mapping,
    eps: float = 1e-15,
) -> pd.DataFrame:
    class_labels = [c.replace("p_", "") for c in proba_cols]
    inv_class_mapping = {v: k for k, v in class_mapping.items()}

    rows: list[dict] = []

    for model, g in df.groupby("model"):
        y_true_int = g["y_true"].map(class_mapping).to_numpy()

        if np.isnan(y_true_int).any():
            bad_labels = (
                g.loc[np.isnan(y_true_int), "y_true"]
                .unique()
                .tolist()
            )
            raise ValueError(f"Unmapped y_true labels found for model {model}: {bad_labels}")

        y_true_int = y_true_int.astype(int)
        y_prob = g[proba_cols].to_numpy()

        # Per-class metrics (one-vs-rest)
        for k in range(len(proba_cols)):
            y_true_bin = (y_true_int == k).astype(int)

            p_k = y_prob[:, k]
            p_k = np.clip(p_k, eps, 1 - eps)

            rows.append(
                {
                    "model": model,
                    "class": inv_class_mapping[k],  # or use class_labels[k]
                    "brier_score": float(brier_score_loss(y_true_bin, p_k)),
                    "logloss": float(log_loss(y_true_bin, np.column_stack([1 - p_k, p_k]), labels=[0, 1])),
                }
            )

    return pd.DataFrame(rows)


metrics_df = brier_and_logloss_per_model_class(df_preds)
OUTPUT_DIR = "MTUM_phase_1_output_distribution"
os.makedirs(OUTPUT_DIR, exist_ok=True)

metrics_df.to_csv(os.path.join(OUTPUT_DIR, "brier_logloss_per_model_class.csv"), index=False)

In [3]:
PALETTE     = ["#4C72B0", "#DD8452", "#55A868", "#C44E52", "#8172B3",
               "#937860", "#DA8BC3", "#8C8C8C", "#CCB974", "#64B5CD"]

def _to_long_probs(df_preds, p_cols, model_col):
    long_df = (
        df_preds[[model_col] + list(p_cols)]
        .melt(id_vars=model_col, var_name="prob_col", value_name="p")
        .dropna(subset=["p"])
        .copy()
    )
    long_df["prob_col"] = pd.Categorical(long_df["prob_col"], categories=list(p_cols), ordered=True)
    return long_df
    
def plot_histograms_all_pcols_by_model(
    df_preds,
    p_cols,
    model_col="model",
    nbins=20,              # was 100
    ncols=4,
    opacity=0.55,
    plot_width=420,
    plot_height=280,
):
    long_df       = _to_long_probs(df_preds, p_cols, model_col)
    models        = sorted(long_df[model_col].unique())
    pcols_ordered = list(long_df["prob_col"].cat.categories)
    color_map     = {m: PALETTE[i % len(PALETTE)] for i, m in enumerate(models)}

    n_plots = len(pcols_ordered)
    nrows   = math.ceil(n_plots / ncols)

    bin_edges = np.linspace(0, 1, nbins + 1)
    bin_width = 1 / nbins

    fig = make_subplots(
        rows=nrows, cols=ncols,
        subplot_titles=list(pcols_ordered) + [""] * (nrows * ncols - n_plots),
        shared_xaxes=True,
        shared_yaxes=False,   # each subplot scales independently
        horizontal_spacing=0.06,
        vertical_spacing=0.09,
    )

    for i, pc in enumerate(pcols_ordered):
        r, c = divmod(i, ncols)
        r, c = r + 1, c + 1
        for m in models:
            g = long_df[(long_df["prob_col"] == pc) & (long_df[model_col] == m)]["p"]
            fig.add_trace(
                go.Histogram(
                    x=g,
                    xbins=dict(start=0, end=1, size=bin_width),
                    opacity=opacity,
                    name=str(m),
                    legendgroup=str(m),
                    showlegend=(i == 0),
                    marker=dict(
                        color=color_map[m],
                        line=dict(color="white", width=0.6),
                    ),
                ),
                row=r, col=c,
            )

    fig.update_layout(
        title=dict(
            text="Predicted probability distributions by class & model",
            font=dict(family="Georgia, serif", size=18, color="#1a1a2e"),
            x=0.5, xanchor="center",
        ),
        barmode="overlay",
        bargap=0.05,
        width=plot_width * ncols,
        height=plot_height * nrows + 140,
        paper_bgcolor="#f8f9fb",
        plot_bgcolor="#ffffff",
        margin=dict(l=70, r=30, t=90, b=90),
        font=dict(family="Arial, sans-serif", size=11, color="#444"),
        legend=dict(
            title=dict(text=model_col, font=dict(size=12)),
            orientation="h",
            yanchor="top", y=-0.04,
            xanchor="center", x=0.5,
            bgcolor="rgba(255,255,255,0.85)",
            bordercolor="#ddd", borderwidth=1,
        ),
    )

    fig.update_xaxes(
        range=[0, 1], tick0=0, dtick=0.25,
        showgrid=True, gridcolor="#e8e8e8", gridwidth=1,
        zeroline=False, linecolor="#ccc",
    )
    fig.update_yaxes(
        showgrid=True, gridcolor="#e8e8e8", gridwidth=1,
        zeroline=False, linecolor="#ccc",
    )

    # y-label only on left edge of each row (col=1 only, since shared_yaxes=False)
    for rr in range(1, nrows + 1):
        fig.update_yaxes(title_text="Count" if rr == math.ceil(nrows / 2) else "", row=rr, col=1)
    for cc in range(1, ncols + 1):
        fig.update_xaxes(title_text="Predicted probability" if cc == math.ceil(ncols / 2) else "", row=nrows, col=cc)

    for ann in fig.layout.annotations:
        ann.update(font=dict(size=12, color="#1a1a2e", family="Georgia, serif"))

    fig.write_html(os.path.join(OUTPUT_DIR, "output_distributions.html"))
    return fig


fig_hist = plot_histograms_all_pcols_by_model(df_preds, p_cols=proba_cols, model_col="model")


NameError: name 'df_preds' is not defined

In [4]:
OUTPUT_DIR = "MTUM_phase_1_output_distribution"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def plot_binned_success_rate_by_model(
    df_preds,
    p_cols,
    model_col="model",
    y_true_col="y_true",
    nbins=10,
    prob_range=(0.0, 1.0),
    ncols=4,
    plot_width=420,
    plot_height=280,
):
    lo, hi = prob_range
    bin_edges = np.linspace(lo, hi, nbins + 1)

    long_df = (
        df_preds[[model_col, y_true_col] + list(p_cols)]
        .melt(id_vars=[model_col, y_true_col], var_name="prob_col", value_name="p")
        .dropna(subset=["p", y_true_col, model_col])
        .copy()
    )
    long_df["prob_col"]    = pd.Categorical(long_df["prob_col"], categories=list(p_cols), ordered=True)
    long_df["class_label"] = long_df["prob_col"].astype(str).str.replace(r"^p_", "", regex=True)
    long_df["success"]     = (long_df[y_true_col].astype(str) == long_df["class_label"]).astype(int)
    long_df["bin_idx"]     = pd.cut(long_df["p"], bins=bin_edges, include_lowest=True, right=True, labels=False)
    long_df = long_df.dropna(subset=["bin_idx"]).copy()
    long_df["bin_idx"] = long_df["bin_idx"].astype(int)

    grouped = (
        long_df
        .groupby([model_col, "prob_col", "bin_idx"], as_index=False, observed=True)
        .agg(n=("p", "size"), avg_p=("p", "mean"), avg_success=("success", "mean"))
    )

    pcols_ordered = list(long_df["prob_col"].cat.categories)
    models        = sorted(grouped[model_col].unique())
    color_map = {m: PALETTE[i % len(PALETTE)] for i, m in enumerate(models)}
    dash_map  = {m: "solid"                    for m in models}

    n_plots = len(pcols_ordered)
    nrows   = math.ceil(n_plots / ncols)

    fig = make_subplots(
        rows=nrows, cols=ncols,
        subplot_titles=list(pcols_ordered) + [""] * (nrows * ncols - n_plots),
        shared_xaxes=True, shared_yaxes=True,
        horizontal_spacing=0.04,
        vertical_spacing=0.09,
    )

    for i, pc in enumerate(pcols_ordered):
        r, c = i // ncols + 1, i % ncols + 1

        # Perfect calibration reference line
        fig.add_trace(
            go.Scatter(
                x=[0, 1], y=[0, 1],
                mode="lines",
                name="Perfect calibration",
                legendgroup="__diag__",
                showlegend=(i == 0),
                line=dict(color="#aaaaaa", dash="dash", width=1.2),
                hoverinfo="skip",
            ),
            row=r, col=c,
        )

        for m in models:
            g = grouped[(grouped["prob_col"] == pc) & (grouped[model_col] == m)].sort_values("avg_p")
            if g.empty:
                continue
            fig.add_trace(
                go.Scatter(
                    x=g["avg_p"], y=g["avg_success"],
                    mode="lines+markers",
                    name=str(m),
                    legendgroup=str(m),
                    showlegend=(i == 0),
                    line=dict(color=color_map[m], dash=dash_map[m], width=2),
                    marker=dict(color=color_map[m], size=5),
                    customdata=np.stack([g["n"]], axis=-1),
                    hovertemplate="avg_p=%{x:.3f}<br>success=%{y:.3f}<br>n=%{customdata[0]}<extra></extra>",
                ),
                row=r, col=c,
            )

    fig.update_layout(
        title=dict(
            text=f"Binned calibration curves by class & model ({nbins} bins)",
            font=dict(family="Georgia, serif", size=18, color="#1a1a2e"),
            x=0.5, xanchor="center",
        ),
        width=plot_width * ncols,
        height=plot_height * nrows + 140,
        paper_bgcolor="#f8f9fb",
        plot_bgcolor="#ffffff",
        margin=dict(l=70, r=30, t=90, b=90),
        font=dict(family="Arial, sans-serif", size=11, color="#444"),
        legend=dict(
            title=dict(text=model_col, font=dict(size=12)),
            orientation="h",
            yanchor="top", y=-0.04,
            xanchor="center", x=0.5,
            bgcolor="rgba(255,255,255,0.85)",
            bordercolor="#ddd", borderwidth=1,
        ),
    )

    fig.update_xaxes(range=[0, 1], tick0=0, dtick=0.25, showgrid=True,
                     gridcolor="#e8e8e8", zeroline=False, linecolor="#ccc")
    fig.update_yaxes(range=[0, 1], tick0=0, dtick=0.25, showgrid=True,
                     gridcolor="#e8e8e8", zeroline=False, linecolor="#ccc")

    for rr in range(1, nrows + 1):
        fig.update_yaxes(title_text="Observed rate" if rr == math.ceil(nrows / 2) else "", row=rr, col=1)
    for cc in range(1, ncols + 1):
        fig.update_xaxes(title_text="Predicted probability" if cc == math.ceil(ncols / 2) else "", row=nrows, col=cc)

    for ann in fig.layout.annotations:
        ann.update(font=dict(size=12, color="#1a1a2e", family="Georgia, serif"))

    fig.write_html(os.path.join(OUTPUT_DIR, "calibration_curves.html"))
    return fig


fig_cal = plot_binned_success_rate_by_model(
    df_preds, p_cols=proba_cols, model_col="model", y_true_col="y_true", nbins=10,
)


NameError: name 'os' is not defined